This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import statistics

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2018-03-01&end_date=2018-03-02&api_key={API_KEY}"
r = requests.get(url)
json_data = r.json()
print(json_data)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2018-03-01', 'end_date': '2018-03-02', 'frequency': 'daily', 'data': [['2018-03-02', None, 49.7, 48.94, 49.02, None, 77271.0, 3803033.0, None, None, None], ['2018-03-01', None, 50.8, 50.05, 50.25, None, 121721.0, 6130791.0, None, None, None]], 'collapse': None, 'order': None}}


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

for key, value in json_data.items():
    print(key + ":", value)

dataset_data: {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2018-03-01', 'end_date': '2018-03-02', 'frequency': 'daily', 'data': [['2018-03-02', None, 49.7, 48.94, 49.02, None, 77271.0, 3803033.0, None, None, None], ['2018-03-01', None, 50.8, 50.05, 50.25, None, 121721.0, 6130791.0, None, None, None]], 'collapse': None, 'order': None}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# **Task 1:** Get the data for AFX from the Nasdaq API for the year 2017 in JSON format
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}"
r = requests.get(url)
json_data = r.json()

In [6]:
# **Task 2:** Convert JSON to Python dictionary
dict_data = dict(json_data)
# print(dict_data) # used to verify dictionary was made correctly

In [7]:
# **Task 3:** Calculate the highest and lowest opening prices for AFX in 2017

# Create a list containing all open prices in 2017. Prices are converted to string format
open_prices_str = [str(list[1]) for list in dict_data["dataset_data"]["data"]]

# Some open prices are missing. The code below will convert these missing prices to the last open price we have data for
for price in open_prices_str:
    if price == "None":
        open_prices_str[open_prices_str.index(price)] = open_prices_str[open_prices_str.index(price) - 1]
        
# Convert the list from strings to floats
open_prices_flt = [float(i) for i in open_prices_str]
        
# print(open_prices_flt) # used to verify list comprehension and data replacement worked correctly

# Use open_prices list to calculate maximum open price for 2017
max_open_price = max(open_prices_flt)
# print(max_open_price) # used to verify max open price calculated correctly

# Use open_prices list to calculate minimum open price for 2017
min_open_price = min(open_prices_flt)
# print(min_open_price) # used to verify min open price calculated correctly

# Print the output for max and min opening price for AFX_X in 2017
print(f"The highest opening price in 2017 for AFX_X was ${max_open_price} and the lowest opening price in 2017 for AFX_X was ${min_open_price}.")

The highest opening price in 2017 for AFX_X was $53.11 and the lowest opening price in 2017 for AFX_X was $34.0.


In [8]:
# **Task 4:** Determine the largest intra-day change based on high and low price

# Create a list of values of the difference between the high price and the low price for each day
intraday_price_changes = [float(list[2] - list[3]) for list in dict_data["dataset_data"]["data"]]
# print(intraday_price_changes) # used to verify list comprehension worked properly

# Calculate the largest intraday price change using intraday_price_changes list and print the result
max_price_change = max(intraday_price_changes)
print(f"The largest intra-day price change for AFX_X in 2017 was ${max_price_change:.2f}.")

The largest intra-day price change for AFX_X in 2017 was $2.81.


In [9]:
# **Task 5:** Determine the largest change in price between any two days based on closing price

# Create a list of values of the closing prices for 2017
closing_prices = [float(list[4]) for list in dict_data["dataset_data"]["data"]]
# print(closing_prices) # Used to verify list comprehension worked correctly

# Calculate difference between closing price of one day and the closing price the day before. Store values in a new list
closing_price_changes = [price - closing_prices[closing_prices.index(price)-1] for price in closing_prices]
closing_price_changes.pop(0) # Need to remove the first item of the list since there is no previous closing price to compare to
# print(closing_price_changes) # Used to verify list was created correctly

# Determine the largest change between closing prices using the max and abs functions on the closing_price_changes list
max_closing_price_change = max([abs(price) for price in closing_price_changes])

# Print the output
print(f"The largest change between closing prices for AFX_X in 2017 over two days was ${max_closing_price_change:.2f}.")

The largest change between closing prices for AFX_X in 2017 over two days was $2.56.


In [10]:
# **Task 6:** Determine the average daily trading volume of AFX_X in 2017

# Create a list of trading volumes for 2017
trading_volumes = [float(list[6]) for list in dict_data["dataset_data"]["data"]]
# print(trading_volumes) # Used to verify list comprehension worked correctly

# Calculate the average trading volume using the trading_volumes list
average_trading_vol = statistics.mean(trading_volumes)

# Print the output
print(f"The average daily trading volume for AFX_X in 2017 was {average_trading_vol:.0f} shares.")

The average daily trading volume for AFX_X in 2017 was 89124 shares.


In [11]:
# **Task 7:** Determine the mediam daily trading volume of AFX_X in 2017

# Calculate the median trading volume using the trading_volumes list
median_trading_vol = statistics.median(trading_volumes)

# Print the output
print(f"The median daily trading volume for AFX_X in 2017 was {median_trading_vol:.0f} shares.")

The median daily trading volume for AFX_X in 2017 was 76286 shares.
